## Heading 

In [ ]:
import datetime
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import boto3
from shapely.geometry import Point
import geopandas as gpd
import folium
from branca.element import Figure
import time

from ipynb.fs.full.qld_fire_extraction import qld_data
from ipynb.fs.full.vic_fire_extraction import vic_data
from ipynb.fs.full.put_to_s3_bucket import put_to_s3_bucket
from ipynb.fs.full.dummy_2 import select_distinct_data
from ipynb.fs.full.dummy_2 import distinct_data_df
from ipynb.fs.full.dummy_2 import points_in_polys_join
from ipynb.fs.full.dummy_2 import popup_html
from ipynb.fs.full.dummy_2 import map_data

aws_region = "ap-southeast-2"
schema_name = "fire_data"
table_name ='rfs_fire_data'
result_output_location = "s3://camgoo2-rfs-visualisation/query_results/"
bucket='camgoo2-rfs-visualisation'
athena_client = boto3.client('athena',region_name=aws_region)
s3_client = boto3.client('s3')


qld_data(datetime,requests,ET,pd)
vic_data(datetime,requests,ET,pd)
put_to_s3_bucket(datetime,s3_client)
execution_id = select_distinct_data(athena_client,result_output_location)
response_path = "query_results/"+execution_id+'.csv'
#allow time for athena query to execute
time.sleep(5) 
fires_df = distinct_data_df(s3_client,bucket,response_path,pd,Point,gpd)
pointInPolys_df = points_in_polys_join(gpd,fires_df)
map_data(pointInPolys_df,folium,pd)
